In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss
from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### A basic methodology for approacing TPG May competition

### Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', sep=',')
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', sep=',')

In [ ]:
train = train.set_index('id')
test = test.set_index('id')


In [ ]:
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv', sep=',')
sub_sample = sub_sample.set_index('id')

In [ ]:
fig = px.histogram(train, x="target",
                   width=600, 
                   height=400,
                   histnorm='percent',
                   template="simple_white"
                   )
fig.update_layout(title="Target Description", 
                  font_family="San Serif",
                  titlefont={'size': 20},
                  showlegend=True,
                  legend=dict(
                      orientation="v", 
                      y=1, 
                      yanchor="top", 
                      x=1.0, 
                      xanchor="right"
                  )                
                 ).update_xaxes(categoryorder='total descending')#
fig.update_traces( 
                  marker_line_width=1.5, opacity=0.99)
fig.show()

### Estimation model Log-loss vs Random Guessing

rewrite a function I've read in an article some time ago..

In [ ]:
def calc_loss(class_perc, num):
    
    lst=[]
    
    for i,z in enumerate(class_perc):
        lst = lst+[i for x in range(int(z*(num+1)))]
        
    preds=[]
    
    for i in range(num):
        preds+=[class_perc]
    
    return (log_loss(lst,preds))

In [ ]:
train['target'].value_counts(normalize=True)

In [ ]:
train['target'].count()

In [ ]:
calc_loss([0.57497, 0.21420, 0.12593, 0.08490], 200000)

In [ ]:
#example of how could be the distribution in classes according to the LB
calc_loss([0.60, 0.20, 0.10, 0.10], 200000)

In [ ]:
#target Map
dict1 = dict(zip(list(train.target.unique()),range(4)))
dict1

In [ ]:
train['target']= train['target'].replace(dict1)

In [ ]:
corr = train.corr()
fig = go.Figure(data= go.Heatmap(z=corr,
                                 x=corr.index.values,
                                 y=corr.columns.values,
                                 zmin=-0.05,
                                 zmax=0.05
                                 )
                )
fig.update_layout(title_text='<b>Correlation Matrix<b>',
                  title_x=0.5,
                  titlefont={'size': 24},
                  width=900, height=800,
                  xaxis_showgrid=False,
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed', 
                  paper_bgcolor=None,
                  )
fig.show()

In [ ]:
cols = train.columns[:-1]
target = train['target']
train = train[cols]

In [ ]:
df = pd.concat([train[cols], test[cols]], axis=0)

In [ ]:
unique_df = pd.DataFrame(df.nunique()).reset_index()
unique_df.columns=['features','count']

fig1 = px.bar(unique_df, y='count', x=cols)

fig1.update_layout(title='Feature cardinality in train+test set',
                  xaxis_title='features',
                  yaxis_title='# unique values',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)
fig1.show()

In [ ]:
(train==0).mean().plot(kind='bar', figsize=(15, 5), title='# zeros on total')

The rule of thumb seems to be: If the skewness is between -0.5 and 0.5, the data are fairly symmetrical. If the skewness is between -1 and – 0.5 or between 0.5 and 1, the data are moderately skewed. If the skewness is less than -1 or greater than 1, the data are highly skewed.

In [ ]:
df.skew().plot(kind='bar', figsize=(15, 5), title='# Skew')
plt.ylim(0,15)

In [ ]:
df2=df
df2[df2<0]=0
df2 = np.log(df2+1)
df2.skew().plot(kind='bar', figsize=(15, 5), title='# skew after log+1 trasformation')
plt.ylim(0,15)

In [ ]:
df_skew = pd.DataFrame(df2.skew()).reset_index()
df_skew.columns=['features','skew']

In [ ]:
df_skew[df_skew['skew']<1.1]

In [ ]:
#first attempt is to consider as numeric features the one with skewness <1.5 and as categorical all the other
n=1.1
low =list(df_skew[df_skew['skew']<=n]['features'])
high = list(df_skew[df_skew['skew']>n]['features'])

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_features='sqrt')
clf = clf.fit(train, target)
features = pd.DataFrame()
features['feature'] = train.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

features.plot(kind='barh', figsize=(25, 25))

In [ ]:
#training set log+1 trasformation
train[train<0]=0
test[test<0]=0
for l in low:
    train[l] = np.log(train[l]+1)
    test[l] = np.log(test[l]+1)

In [ ]:
train[high] = train[high].astype('category')
test[high] = test[high].astype('category')

### Using Optuna with Catboost

In [ ]:
import optuna

In [ ]:
def objective(trial , data = train , target = target):
    train_x , test_x , train_y , test_y = train_test_split(data , target , \
            test_size = 0.028059109276941666 , random_state = 2)

    params = {'iterations':20000,
              'depth': trial.suggest_int("depth", 3, 80),
              'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 0.0001, 25, log=True),
              'bagging_temperature': trial.suggest_float("bagging_temperature", 0, 100),
              'auto_class_weights':trial.suggest_categorical('auto_class_weights', [None,'Balanced','SqrtBalanced']),
              'grow_policy': 'Lossguide',
              'loss_function':trial.suggest_categorical("loss_function", ['MultiClassOneVsAll', 'MultiClass']),
              'bootstrap_type':trial.suggest_categorical("bootstrap_type", ['Poisson']),
              'use_best_model':True,
              'task_type':'GPU', 
              'cat_features':high,
              'learning_rate': trial.suggest_uniform('learning_rate' , 1e-5 , 1.0),
              'max_bin': trial.suggest_int('max_bin', 5, 700),
              'verbose':False,
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 600),
              'subsample': trial.suggest_uniform('subsample' , 1e-5 , 1.0),
              'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 2, 15),
             }
    model = CatBoostClassifier(**params)
    model.fit(train_x , train_y , eval_set = [(test_x , test_y)] , \
             verbose = False)
    preds =  model.predict_proba(test_x)
    lgl = log_loss(test_y , preds)
    return lgl

In [ ]:
#study = optuna.create_study(direction = 'minimize' , study_name = 'cb')
#study.optimize(objective , n_trials = 100)
#print('numbers of the finished trials:' , len(study.trials))
#print('the best params:' , study.best_trial.params)
#print('the best value:' , study.best_value)

### Use isolation Forest to clean the training set

In [ ]:
iso = IsolationForest(n_estimators=500,contamination=0.003, random_state=1)
yhat = iso.fit_predict(train)

In [ ]:
train['anomaly']=yhat
train_clean=train.loc[train['anomaly']!=-1]
train_clean = train_clean.drop(columns=['anomaly'])
train_clean.shape

In [ ]:
train_index=list(train_clean.index)
target_clean = target.iloc[train_index]
target_clean.shape

In [ ]:
param_cb ={
    'depth': 3, 
    'l2_leaf_reg': 4.287566030099442, 
    'bagging_temperature': 27.174417642203863, 
    'auto_class_weights': None, 
    'loss_function': 'MultiClassOneVsAll',
    'eval_metric': 'MultiClassOneVsAll',
    'grow_policy': 'Lossguide',
    'bootstrap_type': 'Poisson',
    'cat_features': high,
    'iterations':10000,
    'max_bin': 484, 
    'min_data_in_leaf': 414,
    'task_type':'GPU',
    'subsample': 0.13534551086578891,
    'max_ctr_complexity':10
}

### Train the Model

using np.clip function to keep probabilities between 0.025–0.975. In this case,there won't be a big growth of the log loss funcion in case of wrong predictions.

In [ ]:
preds = pd.DataFrame(index=test.index)
oof_preds = np.zeros(train_clean.shape[0])
kf = StratifiedKFold(n_splits = 7, random_state = 22 , shuffle = True)
roc = []

n = 0
for trn_idx , val_idx in kf.split(train_clean , target_clean):
    train_x = train_clean.iloc[trn_idx]
    train_y = target_clean.iloc[trn_idx]
    val_x = train_clean.iloc[val_idx]
    val_y = target_clean.iloc[val_idx]
    
    
    model = CatBoostClassifier(**param_cb, random_seed=1)
    model.fit(train_x , train_y , eval_set = [(val_x , val_y)] , verbose = False)
    preds = pd.concat([preds, pd.DataFrame(model.predict_proba(test),index=test.index)], axis=1)
    oof_preds = model.predict_proba(train_clean)
    roc.append(log_loss(val_y ,np.clip(model.predict_proba(val_x),0.025,0.975)))
    
    print(n+1 , roc[n])
    
    n+=1

In [ ]:
dict1

In [ ]:
sub_sample['Class_1']=np.clip(preds[1].mean(axis=1), 0.025,0.975)
sub_sample['Class_2']=np.clip(preds[0].mean(axis=1),0.025,0.975)
sub_sample['Class_3']=np.clip(preds[3].mean(axis=1), 0.025,0.975)
sub_sample['Class_4']=np.clip(preds[2].mean(axis=1), 0.025,0.975)

In [ ]:
sub_sample = sub_sample.reset_index()
sub_sample.head()

In [ ]:
sub_sample.to_csv('submission.csv',index=False)